In [1]:
# !pip install tensorflow

In [2]:
# !pip install opencv-python-headless 
# !pip install pillow  

In [19]:
# !pip install pycocotools

   ---------------------------------------- 0.0/85.8 kB ? eta -:--:--
   ---------------------------------------- 85.8/85.8 kB 4.7 MB/s eta 0:00:00


In [20]:
from pycocotools.coco import COCO

In [21]:
coco = COCO('9mm_annotation.json')

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


In [22]:
coco

ERROR: Could not find a version that satisfies the requirement tensorlow_hub (from versions: none)
ERROR: No matching distribution found for tensorlow_hub


In [28]:
import os

def preprocess_images_in_directory(directory_path):
    processed_images = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".jpg") or filename.endswith(".png"): # Add other file types if necessary
            file_path = os.path.join(directory_path, filename)
            processed_image = preprocess_image(file_path)
            processed_images.append(processed_image)
    return processed_images


images = preprocess_images_in_directory('./images')


In [29]:
images

[array([[0.12941176, 0.18823529, 0.21568627, ..., 0.16078431, 0.15294118,
         0.14509804],
        [0.14901961, 0.20784314, 0.23137255, ..., 0.12941176, 0.1372549 ,
         0.15686275],
        [0.16470588, 0.30980392, 0.39215686, ..., 0.14901961, 0.14901961,
         0.2       ],
        ...,
        [0.14509804, 0.14117647, 0.1372549 , ..., 0.0627451 , 0.05882353,
         0.0745098 ],
        [0.20784314, 0.17254902, 0.1372549 , ..., 0.06666667, 0.05882353,
         0.06666667],
        [0.19607843, 0.18039216, 0.1372549 , ..., 0.0745098 , 0.05882353,
         0.06666667]]),
 array([[0.24313725, 0.25490196, 0.25098039, ..., 0.14117647, 0.14117647,
         0.14901961],
        [0.22352941, 0.23137255, 0.23921569, ..., 0.14117647, 0.13333333,
         0.1372549 ],
        [0.18039216, 0.21960784, 0.25882353, ..., 0.14509804, 0.1372549 ,
         0.1372549 ],
        ...,
        [0.2       , 0.19607843, 0.17647059, ..., 0.1372549 , 0.16862745,
         0.18039216],
        [0.2

In [31]:
# !pip install scikit-image

  Using cached scikit_image-0.22.0-cp311-cp311-win_amd64.whl.metadata (13 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached tifffile-2023.12.9-py3-none-any.whl.metadata (31 kB)
  Using cached lazy_loader-0.3-py3-none-any.whl.metadata (4.3 kB)
Using cached scikit_image-0.22.0-cp311-cp311-win_amd64.whl (24.5 MB)
   ---------------------------------------- 0.0/313.3 kB ? eta -:--:--
   ---------------------------- ----------- 225.3/313.3 kB 4.7 MB/s eta 0:00:01
   ---------------------------------------- 313.3/313.3 kB 4.9 MB/s eta 0:00:00
Using cached lazy_loader-0.3-py3-none-any.whl (9.1 kB)
Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
Using cached tifffile-2023.12.9-py3-none-any.whl (223 kB)


In [33]:
from pycocotools.coco import COCO
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize

# Step 1: Load Annotation File
coco = COCO('9mm_annotation.json')

# Step 2: Prepare the Dataset
class Dataset():
    def __init__(self, annotation_file, image_dir):
        self.coco = COCO(annotation_file)
        self.image_dir = image_dir
        self.image_ids = self.coco.getImgIds()

    def load_image(self, image_id):
        image_info = self.coco.loadImgs(image_id)[0]
        image_path = os.path.join(self.image_dir, image_info['file_name'])
        image = imread(image_path)
        image = resize(image, (224, 224), preserve_range=True)
        image = image.astype(np.float32)
        image /= 255.0
        return image

    def load_mask(self, image_id):
        annotation_ids = self.coco.getAnnIds(imgIds=image_id)
        annotations = self.coco.loadAnns(annotation_ids)
        # Assuming there is only one annotation per image
        annotation = annotations[0]
        mask = self.coco.annToMask(annotation)
        mask = resize(mask, (224, 224), preserve_range=True)
        return mask.astype(np.bool)

# Example usage
dataset = Dataset('9mm_annotation.json', './images')
image = dataset.load_image(dataset.image_ids[0])
mask = dataset.load_mask(dataset.image_ids[0])


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_21608\1549356398.py:33: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return mask.astype(np.bool)


In [36]:
from sklearn.model_selection import train_test_split
train_ids, test_ids = train_test_split(dataset.image_ids, test_size=0.2, random_state=42)

In [39]:
# Create lists to hold your images and annotations
X = []  # This will be a list of image file paths or the images themselves
Y = []  # This will be a list of your annotations (masks)

# Loop through all image_ids to populate X and Y
for image_id in dataset.image_ids:
    # Here we assume that the load_image function gives us the file path
    X.append(dataset.load_image(image_id))
    
    # For Y, we only need the mask data
    mask_id= dataset.load_mask(image_id)
    Y.append(mask_id)

# Now you can split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_21608\1549356398.py:33: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return mask.astype(np.bool)


In [43]:
len(X_test),len(X_train)

(19, 72)

In [44]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

# Define the backbone network (simplified)
input_tensor = Input(shape=(224, 224, 3))
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_tensor)
x = MaxPooling2D((2, 2))(x)
# Add more convolutional layers as needed
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)

# Outputs for RPN (region proposal network)
rpn_class = Dense(1, activation='sigmoid')(x)  # Binary classification for object/non-object
rpn_bbox = Dense(4)(x)  # Bounding box regressor

# Outputs for final detection
final_class = Dense(num_classes, activation='softmax')(x)  # Classification output
final_bbox = Dense(4)(x)  # Bounding box output

# Define the model
model = Model(inputs=input_tensor, outputs=[rpn_class, rpn_bbox, final_class, final_bbox])

# Compile the model
model.compile(optimizer='adam', 
              loss={'rpn_class': 'binary_crossentropy',
                    'rpn_bbox': 'mean_squared_error',
                    'final_class': 'categorical_crossentropy',
                    'final_bbox': 'mean_squared_error'},
              metrics={'rpn_class': 'accuracy',
                       'final_class': 'accuracy'})

# Model summary
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 64  1792        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 112, 112, 64  0           ['conv2d[0][0]']                 
                                )                                                           